In [1]:
import os
import sys
sys.path.append('../mlai_research/')
import log
import utils
import cv2
import numpy as np
from skimage import feature
from skimage.segmentation import felzenszwalb
from skimage.color import rgb2hsv
from skimage.filters import threshold_sauvola
from skimage.segmentation import mark_boundaries
from rasterio.mask import mask
from rasterio.enums import Resampling
from rasterio.warp import reproject, Resampling
from rasterio.plot import show
from shapely.geometry import box, mapping
import geopandas as gpd
import rasterio
import rasterio.plot
import glob
import matplotlib.pyplot as plt
from segment_anything import SegmentAnything

ImportError: cannot import name 'SegmentAnything' from 'segment_anything' (/opt/homebrew/Caskroom/miniforge/base/envs/mlai_rp2/lib/python3.9/site-packages/segment_anything/__init__.py)

In [ ]:
logger = log.get_logger(__name__)
conf = utils.load_config('base')

In [ ]:
def load_rgb_images(image_paths):
    images = []
    for path in image_paths:
        img = cv2.imread(path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
        images.append(img_rgb)
    return np.array(images)

def load_grayscale_images(image_paths):
    images = []
    for path in image_paths:
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        images.append(img)
    return np.array(images)

def plot_cropped_images(images, titles, ncols=3):
    """Plot a list of loaded images.
    
    Args:
        images (list): List of loaded images.
        titles (list): List of titles for the images.
        ncols (int): Number of columns for the subplot grid.
    """
    nrows = len(images) // ncols + (len(images) % ncols > 0) # calculate number of rows

    plt.figure(figsize=(10, 10))
    plt.suptitle('Cropped Images', fontsize=18, y=0.95)

    for n, img in enumerate(images):
        # add a new subplot iteratively using nrows and cols
        ax = plt.subplot(nrows, ncols, n + 1)
        # Plot raster crop
        ax.imshow(img)
        # chart formatting
        ax.set_title(os.path.basename(titles[n]), fontsize=8)
        ax.axis('off')
    plt.show()


def normalize_image(image: np.ndarray) -> np.ndarray:
    """
    Normalizes the pixel values of the input image to the range 0-255.

    Parameters:
    - image (numpy.ndarray): The input image.

    Returns:
    - numpy.ndarray: The normalized image.
    """
    normalized_image = ((image - np.min(image)) / (np.max(image) - np.min(image))) * 255
    return normalized_image.astype(np.uint8)

In [ ]:
rgb_fns = utils.get_filenames("../data/02_intermediate/04_cropped_imgs/", "png", 'rgb')
# chm_fns = utils.get_filenames("../data/02_intermediate/04_cropped_imgs/", "png", 'chm')
rgb_imgs = load_rgb_images(rgb_fns[:9])
# chm_imgs = load_grayscale_images(chm_fns[:-9])
# plot_cropped_images(rgb_imgs[:9], rgb_fns[:9])

In [ ]:
def plot_masked_images(normalized_image, segmented_image, masked_image):
    # Set up the plot with 4 columns
    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12, 4))

    # for i in range(num_images):
    # Plot the normalized image in the first column
    axes[0].imshow(normalized_image)
    axes[0].set_title('Normalized Image')

    # Plot the processed image with boundaries in the third column
    axes[1].imshow(mark_boundaries(normalized_image, segmented_image))
    axes[1].set_title('Segmented Image')

    # Plot the masked image in the fourth column
    axes[2].imshow(masked_image)
    axes[2].set_title('Masked Image')

    # Adjust layout to prevent clipping of titles
    plt.tight_layout()
    plt.show()

def create_center_segment_mask(segmentation, image_shape):
    # Step 1: Identify the center point of the image
    center_point = (image_shape[0] // 2, image_shape[1] // 2)

    # Step 2: Identify the segment label at the center point
    center_segment_label = segmentation[center_point]

    # Step 3: Create a mask by comparing the segmentation array with the center segment label
    mask = segmentation == center_segment_label

    # Step 4: Zero out all other segments by multiplying the original image with the mask
    # This step would be done outside this function, when you have the original image available

    return mask


def color_based_segment_mask(segments, image):
    # Convert segments to a binary image
    mask = (segments > 0).astype('uint8')

    # Calculate the average color of each segment
    avg_colors = cv2.mean(image, mask=mask)

    # Convert the average colors to a numpy array of the same size as the color boundaries
    avg_colors = np.array(avg_colors[:3])

    # Define the color range for bushes or shrubs
    lower_color = np.array([25., 50., 50.])
    upper_color = np.array([85., 255., 255.])

    # Keep the segments that fall within the color range
    mask = cv2.inRange(avg_colors, lower_color, upper_color)

    return mask


def texture_based_segment_mask(segments, image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Calculate the Local Binary Pattern of the image
    lbp = feature.local_binary_pattern(gray, P=8, R=1)
    
    # Convert segments_fz to a binary image
    mask = (segments > 0).astype('uint8') * 255

    # Now use this mask in cv2.mean
    avg_lbp = cv2.mean(lbp, mask=mask)
    # # Calculate the average LBP of each segment
    # avg_lbp = cv2.mean(lbp, mask=segments)

    # Define the LBP range for bushes or shrubs
    lower_lbp = 0.2
    upper_lbp = 0.8

    # Keep the segments that fall within the LBP range
    mask = (avg_lbp >= lower_lbp) & (avg_lbp <= upper_lbp)

    return mask

def apply_mask(image, mask):
    # Ensure mask is boolean
    mask = mask.astype(bool)
    
    # Expand dimensions of the mask to match the image
    mask = np.expand_dims(mask, axis=-1)
    
    # Apply the mask to the image
    masked_image = image * mask
    
    return masked_image

def apply_color_mask(image, mask):
    # Ensure mask is boolean
    mask = mask.astype(bool)
    
    # Repeat the mask along its missing dimensions
    # Expand dimensions of the mask to match the image
    mask = np.expand_dims(mask, axis=-1)
    mask = np.repeat(mask, image.shape[-1], axis=-1)
    
    # Apply the mask to the image
    masked_image = image * mask
    
    return masked_image

In [ ]:
rgb_img = normalize_image(rgb_imgs[3])
segments_fz = felzenszwalb(rgb_img, scale=150, sigma=0.5, min_size=100)
# mask = create_center_segment_mask(segments_fz, rgb_img.shape)
mask = color_based_segment_mask(segments_fz, rgb_img)
masked_image = apply_color_mask(rgb_img, mask)
# mask = texture_based_segment_mask(segments_fz, rgb_img)
# masked_image = apply_mask(rgb_img, mask)
plot_masked_images(rgb_img, segments_fz, masked_image)

In [ ]:
# Convert segments to a binary image
mask = (segments_fz > 0).astype('uint8')

# Calculate the average color of each segment
avg_colors = cv2.mean(rgb_img, mask=mask)

In [ ]:
avg_colors

In [ ]:
avg_colors = np.array(avg_colors[:3])

In [ ]:
avg_colors

In [ ]:
lower_color = np.array([25., 50., 50.])
upper_color = np.array([85., 255., 255.])

In [ ]:
lower_color

In [ ]:
mask = cv2.inRange(avg_colors, lower_color, upper_color)


In [ ]:
def load_and_plot_chm(chm_path):
    """Load and plot a Canopy Height Model (CHM) from a TIFF file.
    
    Args:
        chm_path (str): Path to the CHM TIFF file.
    """
    # Open the file using rasterio
    with rasterio.open(chm_path) as chm_dataset:
        # Read the CHM data into a 2D array
        chm_data = chm_dataset.read(1)
        
    # Plot the CHM data
    plt.figure(figsize=(10, 10))
    rasterio.plot.show(chm_data, cmap='gray')
    plt.title('Canopy Height Model')
    plt.show()

In [ ]:
dsm_path = '../data/02_intermediate/03_cropped_tifs/283_rgb_Xanthium.tif'

In [ ]:
def normalize_image(image: np.ndarray) -> np.ndarray:
    """
    Normalizes the pixel values of the input image.

    Parameters:
    - image (numpy.ndarray): The input image.

    Returns:
    - numpy.ndarray: The normalized image.
    """
    normalized_image = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    return normalized_image

def load_dsm_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load the image in grayscale mode
    return img

def plot_dsm_image(image):
    plt.imshow(image, cmap='gray')  # Use a grayscale colormap
    plt.show()

In [ ]:
dsm_image = load_dsm_image(dsm_path)
normalized_dsm_image = normalize_image(dsm_image)
plot_dsm_image(normalized_dsm_image)

In [ ]:
with rasterio.open(chm_path) as dst:
        # Read the CHM data into a 2D array
        dsm_data = dst.read(1)

In [ ]:
min_value = np.min(dsm_data)
max_value = np.max(dsm_data)
mean_value = np.mean(dsm_data)
min_value, max_value, mean_value

In [ ]:
# Normalize the data to 0-1 range
dsm_data_norm = (dsm_data - np.min(dsm_data)) / (np.max(dsm_data) - np.min(dsm_data))

# Plot the normalized data
plt.figure(figsize=(10, 6))
plt.imshow(dsm_data_norm, cmap='gray')
plt.colorbar(label='Normalized Values')
plt.title('Normalized DSM Data')
plt.show()

In [ ]:

load_and_plot_chm(chm_path)

In [ ]:
chm_img = rasterio.open(f"../data/02_intermediate/03_cropped_tifs/279_chm_Xanthium.tif") 

In [ ]:
chm_data = chm_img.read(1)
chm_data.shape

In [ ]:
plt.figure(figsize=(10, 10))
rasterio.plot.show(chm_data, cmap='viridis')
plt.title('Canopy Height Model')
plt.show()

In [ ]:
chm_img

In [ ]:
min_value = np.min(chm_img)
max_value = np.max(chm_img)
min_value, max_value

In [ ]:
def load_cropped_tifs(path):
    return glob.glob(f'{path}*.tif')

def normalize_image(image: np.ndarray) -> np.ndarray:
    """
    Normalizes the pixel values of the input image.

    Parameters:
    - image (numpy.ndarray): The input image.

    Returns:
    - numpy.ndarray: The normalized image.
    """
    normalized_image = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    logger.info(f'Normalized image shape: {normalized_image.shape}')
    return normalized_image

def process_rgb_image(image: np.ndarray) -> np.ndarray:
    """
    Extracts the RGB channels from the input image.

    Parameters:
    - image (numpy.ndarray): The input image.

    Returns:
    - numpy.ndarray: The RGB image.
    """
    # Add your RGB processing logic here
    img_rgb = image[:, :, :3]
    logger.info(f'RGB image shape: {img_rgb.shape}')
    return img_rgb


def get_segments(image: np.ndarray) -> np.ndarray:
    """
    Performs segmentation on the input image using the Felzenszwalb algorithm.

    Parameters:
    - image (numpy.ndarray): The input RGB image.

    Returns:
    - numpy.ndarray: The segmented image.
    """
    # Add your segmentation logic here
    segments_fz = felzenszwalb(image, scale=100, sigma=0.5, min_size=50)
    return segments_fz

# def segment_image(image: np.ndarray) -> np.ndarray:
#     """
#     Segments the input image to extract plant regions based on color.

#     Parameters:
#     - image (numpy.ndarray): The input RGB image.

#     Returns:
#     - numpy.ndarray: The segmented image containing only plant regions.
#     """
#     # Convert the image from RGB to HSV
#     hsv_image = rgb2hsv(image)
    
#     # Define the color range for the plants
#     lower_green = np.array([35/360, 0.2, 0.2])
#     upper_green = np.array([85/360, 1, 1])
    
#     # Create a mask for the plant segments
#     plant_mask = cv2.inRange(hsv_image, lower_green, upper_green)
    
#     # Apply the mask to the image
#     plant_segments = cv2.bitwise_and(image, image, mask=plant_mask)
    
#     return plant_segments

# def segment_image(image: np.ndarray) -> np.ndarray:
#     """
#     Segments the input image to extract plant regions using adaptive thresholding.

#     Parameters:
#     - image (numpy.ndarray): The input RGB image.

#     Returns:
#     - numpy.ndarray: The segmented image containing only plant regions.
#     """
#     # Convert the image from RGB to HSV
#     hsv_image = rgb2hsv(image)

#     # Convert the HSV image to grayscale
#     gray_image = cv2.cvtColor(hsv_image, cv2.COLOR_BGR2GRAY)

#     # Convert the grayscale image to 8-bit unsigned integer
#     gray_image = np.uint8(gray_image)

#     # Apply adaptive thresholding (Otsu's method)
#     _, plant_mask = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

#     # Create a mask with the same number of channels as the input image
#     plant_mask_rgb = cv2.cvtColor(plant_mask, cv2.COLOR_GRAY2RGB)

#     # Ensure the mask has the correct data type and size
#     plant_mask_rgb = np.uint8(plant_mask_rgb)

#     # Apply the mask to the original image
#     plant_segments = cv2.bitwise_and(image, image, mask=plant_mask_rgb)

#     return plant_segments

def rgb2hsv(image: np.ndarray) -> np.ndarray:
    """
    Converts an RGB image to HSV.

    Parameters:
    - image (numpy.ndarray): The input RGB image.

    Returns:
    - numpy.ndarray: The converted HSV image.
    """
    return cv2.cvtColor(image, cv2.COLOR_RGB2HSV)


def segment_image(image: np.ndarray) -> np.ndarray:
    """
    Segments the input image to extract plant regions using adaptive thresholding.

    Parameters:
    - image (numpy.ndarray): The input RGB image.

    Returns:
    - numpy.ndarray: The segmented image containing only plant regions.
    """
    # Convert the image from RGB to HSV
    hsv_image = rgb2hsv(image)

    # # Extract the V channel from the HSV image
    # v_channel = hsv_image[:, :, 2]

    # Convert the HSV image to grayscale
    gray_image = cv2.cvtColor(hsv_image, cv2.COLOR_BGR2GRAY)

    # Scale the grayscale image from [0, 1] to [0, 255] and convert to uint8
    gray_image = (gray_image * 255).astype(np.uint8)

    # Apply adaptive thresholding (Gaussian method)
    plant_mask = cv2.adaptiveThreshold(gray_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)

    # Create a mask with the same number of channels as the input image
    plant_mask_rgb = cv2.cvtColor(plant_mask, cv2.COLOR_GRAY2RGB)

    # Ensure the mask has the correct data type and size
    # plant_mask_rgb = np.uint8(plant_mask_rgb)

    # Apply the mask to the original image
    plant_segments = cv2.bitwise_and(image, image, mask=plant_mask_rgb)

    return plant_segments


def threshold_image(image: np.ndarray) -> np.ndarray:
    """
    Applies thresholding to the input image.

    Parameters:
    - image (numpy.ndarray): The input image.

    Returns:
    - numpy.ndarray: The binary image after thresholding.
    """
    # Add your thresholding logic here
    thresh_sauvola = threshold_sauvola(image)
    binary_sauvola = image > thresh_sauvola
    return binary_sauvola

# def apply_mask_to_segments(image, segments, plant_segment_ids):
#     masked_image = np.zeros_like(image)
#     for segment_id in plant_segment_ids:
#         # Get the pixels of the segment
#         segment_mask = (segments == segment_id)
        
#         # Apply the mask to the segment and store the result in the masked image
#         masked_image[segment_mask] = image[segment_mask]
    
#     return masked_image

def apply_mask_to_segments(image: np.ndarray, segments: np.ndarray, 
                            plant_segment_ids: list) -> np.ndarray:
    """
    Applies a mask to specific segments in the input image.

    Parameters:
    - image (numpy.ndarray): The input image.
    - segments (numpy.ndarray): The segmentation map.
    - plant_segment_ids (list): List of segment IDs corresponding to plant regions.

    Returns:
    - numpy.ndarray: The masked image containing only the specified plant segments.
    """
    masked_image = np.zeros_like(image)
    
    if len(plant_segment_ids) == 0:  # If no plant segments are found
        # Calculate the sum of pixel values in each segment
        segment_sums = [np.sum(image[segments == segment_id]) for segment_id in np.unique(segments)]
        # Find the segment with the highest sum
        max_segment_id = np.argmax(segment_sums)
        # Add this segment to the plant_segment_ids
        plant_segment_ids.append(max_segment_id)
    
    for segment_id in plant_segment_ids:
        segment_mask = (segments == segment_id)
        masked_image[segment_mask] = image[segment_mask]
    
    return masked_image

def load_raster(image_path):
    with rasterio.open(image_path) as src:
        # Read the data and transpose the dimensions
        raster_data = src.read().transpose(1, 2, 0)
    return raster_data

def process_images(image_paths):
    normalized_images = []
    rgb_images = []
    imgs_segments = []
    masked_images = []

    for image_path in image_paths:
        raster_data = load_raster(image_path)
        logger.info(f'Raster data shape: {raster_data.shape}')

        normalized_image = normalize_image(raster_data)
        normalized_images.append(normalized_image)

        rgb_image = process_rgb_image(normalized_image)
        rgb_images.append(rgb_image)

        segments = get_segments(rgb_image)
        imgs_segments.append(segments)

        plant_segments = segment_image(rgb_image)

        # Replace 'plant_segment_ids' with the list of plant segment IDs
        masked_image = apply_mask_to_segments(rgb_image, segments, [1])
        masked_images.append(masked_image)

    return normalized_images, rgb_images, imgs_segments, masked_images

# def plot_images(normalized_images, rgb_images, segmented_images, masked_images):
#     num_images = len(normalized_images)

#     # Set up the plot with 4 columns
#     fig, axes = plt.subplots(nrows=num_images, ncols=4, figsize=(16, 4*num_images))

#     for i in range(num_images):
#         # Plot the normalized image in the first column
#         axes[i, 0].imshow(normalized_images[i])
#         axes[i, 0].set_title('Normalized Image')

#         # Plot the processed RGB image in the second column
#         axes[i, 1].imshow(rgb_images[i])
#         axes[i, 1].set_title('Processed RGB')

#         # Plot the processed image with boundaries in the third column
#         axes[i, 2].imshow(mark_boundaries(rgb_images[i], segmented_images[i]))
#         axes[i, 2].set_title('Segmented Image')

#         # Plot the masked image in the fourth column
#         axes[i, 3].imshow(masked_images[i])
#         axes[i, 3].set_title('Masked Image')

#     # Adjust layout to prevent clipping of titles
#     plt.tight_layout()
#     plt.show()

def plot_all_rasters(normalized_images, rgb_images, segmented_images, masked_images):
    num_images = len(normalized_images)

    # Set up the plot with 4 columns
    fig, axes = plt.subplots(nrows=num_images, ncols=4, figsize=(16, 4*num_images))

    for i in range(num_images):
        # Plot the normalized image in the first column
        axes[i, 0].imshow(normalized_images[i])
        axes[i, 0].set_title('Normalized Image')

        # Plot the processed RGB image in the second column
        axes[i, 1].imshow(rgb_images[i])
        axes[i, 1].set_title('Processed RGB')

        # Plot the processed image with boundaries in the third column
        axes[i, 2].imshow(mark_boundaries(rgb_images[i], segmented_images[i]))
        axes[i, 2].set_title('Segmented Image')

        # Plot the masked image in the fourth column
        axes[i, 3].imshow(masked_images[i])
        axes[i, 3].set_title('Masked Image')

    # Adjust layout to prevent clipping of titles
    plt.tight_layout()
    plt.show()


def plot_images(normalized_image, segmented_image, masked_image):
    # Set up the plot with 4 columns
    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(16, 3))

    # for i in range(num_images):
    # Plot the normalized image in the first column
    axes[0].imshow(normalized_image)
    axes[0].set_title('Normalized Image')

    # Plot the processed RGB image in the second column
    # axes[1].imshow(rgb_image)
    # axes[1].set_title('Processed RGB')

    # Plot the processed image with boundaries in the third column
    axes[1].imshow(mark_boundaries(normalized_image, segmented_image))
    axes[1].set_title('Segmented Image')

    # Plot the masked image in the fourth column
    axes[2].imshow(masked_image)
    axes[2].set_title('Masked Image')

    # Adjust layout to prevent clipping of titles
    plt.tight_layout()
    plt.show()

In [ ]:
cropped_fns_rgb = load_cropped_tifs(conf.data.path_pri_rgb)

In [ ]:
raster_img = load_raster(cropped_fns_rgb[0])
logger.info(f'Raster data shape: {raster_img.shape}')

In [ ]:
rgb_img = process_rgb_image(raster_img)

In [ ]:
rgb_norm_img = normalize_image(rgb_img)
rgb_not_norm_img = cv2.normalize(rgb_norm_img, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

In [ ]:
segments = get_segments(rgb_norm_img)

In [ ]:
# # Assuming 'image' is your ndarray
min_value = np.min(rgb_img)
max_value = np.max(rgb_img)
min_value, max_value

In [ ]:
# def segment_image(image: np.ndarray) -> np.ndarray:
#     """
#     Segments the input image to extract plant regions using adaptive thresholding.

#     Parameters:
#     - image (numpy.ndarray): The input RGB image.

#     Returns:
#     - numpy.ndarray: The segmented image containing only plant regions.
#     """
#     # Convert the image from RGB to HSV
#     hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

#     # Extract the V channel from the HSV image
#     v_channel = hsv_image[:, :, 2]

#     # Scale the V channel from [0, 1] to [0, 255] and convert to uint8
#     v_channel = (v_channel * 255).astype(np.uint8)

#     # Apply adaptive thresholding (Gaussian method)
#     plant_mask = cv2.adaptiveThreshold(v_channel, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 11, 2)

#     # Create a mask with the same number of channels as the input image
#     plant_mask_rgb = cv2.cvtColor(plant_mask, cv2.COLOR_GRAY2RGB)

#     # Ensure the mask has the correct data type and size
#     plant_mask_rgb = np.uint8(plant_mask_rgb)

#     # Apply the mask to the original image
#     plant_segments = cv2.bitwise_and(image, image, mask=plant_mask_rgb)

#     return plant_segments

# def segment_image(image: np.ndarray) -> np.ndarray:
#     """
#     Segments the input image to extract plant regions using adaptive thresholding.

#     Parameters:
#     - image (numpy.ndarray): The input RGB image.

#     Returns:
#     - numpy.ndarray: The segmented image containing only plant regions.
#     """
#     # Convert the image from RGB to HSV
#     hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

#     # Extract the V channel from the HSV image
#     v_channel = hsv_image[:, :, 2]

#     # Apply adaptive thresholding (Gaussian method)
#     plant_mask = cv2.adaptiveThreshold(v_channel, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 11, 2)

#     # Apply the mask to the original image
#     plant_segments = cv2.bitwise_and(image, image, mask=plant_mask)

#     return plant_segments


def segment_image(image: np.ndarray) -> np.ndarray:
    """
    Segments the input image to extract plant regions using adaptive thresholding and morphological operations.

    Parameters:
    - image (numpy.ndarray): The input RGB image.

    Returns:
    - numpy.ndarray: The segmented image containing only plant regions.
    """
    # Convert the image from RGB to HSV
    hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

    # Extract the V channel from the HSV image
    v_channel = hsv_image[:, :, 2]

    # Apply adaptive thresholding (Gaussian method)
    plant_mask = cv2.adaptiveThreshold(v_channel, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 11, 2)

    # Define a kernel for morphological operations
    kernel = np.ones((3, 3), np.uint8)

    # Apply morphological opening to remove noise
    plant_mask = cv2.morphologyEx(plant_mask, cv2.MORPH_OPEN, kernel)

    # Apply morphological closing to fill small holes and smooth boundaries
    plant_mask = cv2.morphologyEx(plant_mask, cv2.MORPH_CLOSE, kernel)

    # Apply the mask to the original image
    plant_segments = cv2.bitwise_and(image, image, mask=plant_mask)

    return plant_segments

In [ ]:
plant_segments = segment_image(rgb_not_norm_img)

In [ ]:
plot_images(rgb_norm_img, segments, plant_segments)

In [ ]:
plot_images(rgb_norm_img, segments, plant_segments)

In [ ]:



# normalized_images.append(normalized_image)


# rgb_images.append(rgb_image)


# imgs_segments.append(segments)

plant_segments = segment_image(rgb_image)

# Replace 'plant_segment_ids' with the list of plant segment IDs
masked_image = apply_mask_to_segments(rgb_image, segments, [1])
# masked_images.append(masked_image)

In [ ]:
normalized_images, rgb_images, imgs_segments, masked_images = process_images(cropped_fns_rgb[:10])

In [ ]:
plot_images(normalized_images, rgb_images, imgs_segments, masked_images)

In [ ]:
from skimage.measure import regionprops, regionprops_table
import pandas as pd
import numpy as np

def extract_geometric_features(image):
    """
    Extract geometric features from a binary image of a leaf.
    
    Parameters:
    image (numpy.ndarray): Binary image of a leaf.
    
    Returns:
    pandas.DataFrame: DataFrame containing the geometric features.
    """
    props = regionprops(image)
    features = pd.DataFrame(regionprops_table(image, properties=('area', 'perimeter', 'eccentricity', 'extent')))
    features['aspect_ratio'] = props[0].major_axis_length / props[0].minor_axis_length
    features['roundness'] = 4 * np.pi * features['area'] / (features['perimeter'] ** 2)
    features['compactness'] = features['area'] / props[0].convex_area
    return features

In [ ]:
# Initialize arrays to store the color features
mean_colors = np.zeros((segments_fz.max() + 1, 3))  # for mean
std_colors = np.zeros((segments_fz.max() + 1, 3))  # for standard deviation

# Loop over each segment
for segment_id in np.unique(segments_fz):
    # Get the pixels of the segment
    segment_pixels = img[segments_fz == segment_id]
    # Calculate and store the mean and standard deviation of the RGB values of the segment
    mean_colors[segment_id] = segment_pixels.mean(axis=0)
    std_colors[segment_id] = segment_pixels.std(axis=0)

In [ ]:
# segments_qs = quickshift(img_rgb, ratio=0.5, kernel_size=3, max_dist=6, sigma=0)

In [ ]:
# fig, ax = plt.subplots(figsize = (20,20))
# plt.imshow(mark_boundaries(img_rgb, segments_qs))
# plt.show()